некоторые начальные данные

In [1]:
# некоторые начальные данные
table1 = {'Должник': ['Лицо 01', 'Лицо 02', 'Лицо 02', 'Лицо 02', 'Лицо 04'], 
          'Кредитор': ['Лицо 02', 'Лицо 01', 'Лицо 04', 'Лицо 04', 'Лицо 01'],
          'Cумма долга': [500, 200, 150, 250, 100]}

#### я буду использовать библиотеку pandas для дальнейшей работы

In [2]:
import pandas as pd
import random

In [3]:
debts = pd.DataFrame(table1)
debts

,Должник,Кредитор,Cумма долга
0,Лицо 01,Лицо 02,500
1,Лицо 02,Лицо 01,200
2,Лицо 02,Лицо 04,150
3,Лицо 02,Лицо 04,250
4,Лицо 04,Лицо 01,100


сгруппирую идентичные строки

In [4]:
first_groupby = debts.groupby(['Должник', 'Кредитор']).sum().reset_index()
first_groupby 

,Должник,Кредитор,Cумма долга
0,Лицо 01,Лицо 02,500
1,Лицо 02,Лицо 01,200
2,Лицо 02,Лицо 04,400
3,Лицо 04,Лицо 01,100


Теперь необходимо поработать со взаиморасчетами между одними и теми же лицами.  
Я копирую таблицу и меняю в ней местами столбцы (а затем имена столбцов возвращаю. Полученную сумму долга я умножаю на -1. Можно представить, что это "уход денежных средств"

In [5]:
# теперь необходимо поработать со взаиморасчетами между одними и теми же лицами

extra_debts = first_groupby.copy(deep=False)
extra_debts.rename(columns = {'Должник' : 'Кредитор', 'Кредитор' : 'Должник'}, inplace = True) 
extra_debts = extra_debts.reindex(columns=['Должник','Кредитор','Cумма долга'])
extra_debts['Cумма долга'] = extra_debts['Cумма долга'].apply(lambda x: x* (-1))
extra_debts_groupby = extra_debts.groupby(['Должник', 'Кредитор']).sum().reset_index()
extra_debts_groupby 

,Должник,Кредитор,Cумма долга
0,Лицо 01,Лицо 02,-200
1,Лицо 01,Лицо 04,-100
2,Лицо 02,Лицо 01,-500
3,Лицо 04,Лицо 02,-400


Теперь дело остается за малым - объединить две полученные таблицы и применить группировку. Оставшиеся отрицательные числа нам не понадобятся, ведь это не уже не долг. Убираю их. И для красоты обновляю индексирование. 

In [6]:
final_result = pd.concat([first_groupby , extra_debts_groupby]).groupby(['Должник', 'Кредитор']).sum().reset_index()
final_result.drop(final_result[final_result['Cумма долга'] < 0].index, inplace = True)
final_result.reset_index(inplace = True)
final_result.drop(columns='index', inplace = True)
final_result

,Должник,Кредитор,Cумма долга
0,Лицо 01,Лицо 02,300
1,Лицо 02,Лицо 04,400
2,Лицо 04,Лицо 01,100


Полученное решение дало верный результат. Для надгладности, сгенерирую побольше данных и использую уже их, а результат обработки запишу в файл _report.csv_

In [7]:
# генерация данных
list_of_people = []
for i in range(1,15):
    list_of_people.append('Лицо 0' + str(i)) if i < 10 else list_of_people.append('Лицо ' + str(i))

res = []
for _ in range(200):
    res.append(random.choices(list_of_people, k=2))
res = list(filter(lambda x: x[0] != x[1], res))
for i in res:   
    i.append(random.randrange(10, 1000, 25))

In [8]:
debts_2 = pd.DataFrame(res, columns = ['Должник', 'Кредитор', 'Cумма долга'])
debts_2.to_csv('./data')
debts_2.head()

,Должник,Кредитор,Cумма долга
0,Лицо 02,Лицо 13,285
1,Лицо 11,Лицо 01,760
2,Лицо 05,Лицо 04,610
3,Лицо 04,Лицо 08,235
4,Лицо 11,Лицо 12,985


In [9]:
# 1
first_groupby = debts_2.groupby(['Должник', 'Кредитор']).sum().reset_index()
# 2
extra_debts = first_groupby.copy(deep=False)
extra_debts.rename(columns = {'Должник' : 'Кредитор', 'Кредитор' : 'Должник'}, inplace = True) 
extra_debts = extra_debts.reindex(columns=['Должник','Кредитор','Cумма долга'])
extra_debts['Cумма долга'] = extra_debts['Cумма долга'].apply(lambda x: x* (-1))
extra_debts_groupby = extra_debts.groupby(['Должник', 'Кредитор']).sum().reset_index()
# 3
final_result = pd.concat([first_groupby , extra_debts_groupby]).groupby(['Должник', 'Кредитор']).sum().reset_index()
final_result.drop(final_result[final_result['Cумма долга'] <= 0].index, inplace = True)
final_result.reset_index(inplace = True)
final_result.drop(columns='index', inplace = True)

In [10]:
final_result.to_csv('./report', index_label='№')
final_result

,Должник,Кредитор,Cумма долга
0,Лицо 01,Лицо 08,710
1,Лицо 01,Лицо 10,260
2,Лицо 01,Лицо 13,1845
3,Лицо 01,Лицо 14,185
4,Лицо 02,Лицо 01,775
...,...,...,...
74,Лицо 14,Лицо 05,150
75,Лицо 14,Лицо 06,910
76,Лицо 14,Лицо 07,620
77,Лицо 14,Лицо 11,385
